In [1]:
from skilift import GTFS, get_stop_node, read_osm, get_elevations_for_nodes, get_graph_nodes, ElevationAwareStreetDataset

In [2]:
import pandas as pd
from zipfile import ZipFile
from collections import defaultdict
from typing import Dict, Set, Tuple
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
fn = "/home/bmander/skilift_data/transit/redding.zip"

feed = GTFS(fn)

In [4]:
# get epoch time on Aptil 19, 2023 at 1:00pm pacific time
t0 = pd.Timestamp("2023-04-19 13:00:00", tz="America/Los_Angeles")
t0

Timestamp('2023-04-19 13:00:00-0700', tz='America/Los_Angeles')

In [5]:
node0 = get_stop_node(feed, "Downtown Transit Center", t0)
node0

AtStopNode(stop_id:2000, datetime:2023-04-19 13:00:00-07:00)

In [6]:
node0.outgoing[0].node

DepartureNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [7]:
node0.outgoing[0].node.outgoing[0].node

ArrivalNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00)

In [8]:
node0.outgoing[0].node.outgoing[0].node.outgoing

[Edge(node=DepartureNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00), weight=0.0),
 Edge(node=AtStopNode(stop_id:8003, datetime:2023-04-19 14:35:00-07:00), weight=60.0)]

In [9]:
osm_filename = "/home/bmander/skilift_data/street/redding.pbf"


In [10]:
elev_geotiff_filename = "/home/bmander/skilift_data/elevation/USGS_13_n41w123_20210624.tif"

In [11]:
elev_dataset = ElevationAwareStreetDataset(osm_filename, elev_geotiff_filename)

Reading OSM file...done
Indexing ways...done
Getting node elevations...done


In [12]:
from shapely.strtree import STRtree
from shapely.geometry import Point

In [14]:
def geodesic_distance_meters(geo_pt: Point, geo_pt2: Point) -> float:
    """Compute the geodesic distance in meters between two points on the earth's surface.
    """

    # TODO: this can be vectorized if it ever becomes a bottleneck

    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [geo_pt.x, geo_pt.y, geo_pt2.x, geo_pt2.y])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    earth_radius_km = 6371
    km = earth_radius_km * c
    return km * 1000

In [15]:
nd_points = [Point(lon, lat) for lon, lat in elev_dataset.nodes.values()]

In [16]:
tree = STRtree( nd_points )

In [17]:
lon = sum([pt.x for pt in nd_points])/len(nd_points)
lat = sum([pt.y for pt in nd_points])/len(nd_points)
avg = Point(lon, lat)

In [18]:
for nd_ix in tree.query(avg.buffer(0.0003)):
    pt = nd_points[nd_ix]

    # find distance in meters between geographical points
    dist = geodesic_distance_meters(avg, pt)
    print(dist)

33.586155638010396
30.45918994893321
30.448305567346534
30.984222752860745
19.906777976196068


In [20]:
def cons(ary: Iterable) -> Iterator[Tuple[Any, Any]]:
    """Return a generator of consecutive pairs from the input iterable."""
    it = iter(ary)
    prev = next(it)
    for item in it:
        yield prev, item
        prev = item

for way_id, way in elev_dataset.ways.items():

{10556271: Way(nds=[91110929, 8262595439, 9931652586, 8357230294, 7247169143, 7229384797, 91110932, 8356670681, 7247169144, 91110935, 6652605782, 91110936], tags={'highway': 'residential', 'name': 'Lincoln Street', 'surface': 'asphalt'}),
 10556272: Way(nds=[91110941, 91110942], tags={'highway': 'residential', 'name': 'Lincoln Street', 'tiger:cfcc': 'A41', 'tiger:county': 'Shasta, CA', 'tiger:name_base': 'Lincoln', 'tiger:name_type': 'St', 'tiger:reviewed': 'no'}),
 10556273: Way(nds=[91110946, 91110949, 91110955, 6701026090, 91110956, 91110959, 6701026091, 91110961, 6701026092, 8357330729, 91110963, 6700528736, 8357330726, 91110967], tags={'highway': 'residential', 'name': 'Lincoln Street', 'surface': 'asphalt'}),
 10556274: Way(nds=[91110979, 5878549678, 9686400239, 9686400227, 91110981, 91110982, 91110984], tags={'highway': 'residential', 'name': 'Privet Drive', 'tiger:cfcc': 'A41', 'tiger:county': 'Shasta, CA', 'tiger:name_base': 'Privet', 'tiger:name_type': 'Dr', 'tiger:reviewed':